In [5]:
import zipfile
import os

# Define paths
zip_path = "/content/dataset.zip"
extract_path = "/content/extracted_files"

# Extract ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Extracted to: {extract_path}")


Extracted to: /content/extracted_files


In [8]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


mask_path = "/content/extracted_files/dataset/with_mask"
no_mask_path = "/content/extracted_files/dataset/without_mask"

IMG_SIZE = (64, 64)

# Function to extract HOG features
def extract_hog_features(image):
    image = cv2.resize(image, IMG_SIZE)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, _ = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9, visualize=True)
    return fd

# Function to extract LBP features
def extract_lbp_features(image):
    image = cv2.resize(image, IMG_SIZE)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 11), density=True)
    return hist

def load_dataset():
    X, y = [], []

    for img_name in os.listdir(mask_path):
        img = cv2.imread(os.path.join(mask_path, img_name))
        if img is not None:
            hog_features = extract_hog_features(img)
            lbp_features = extract_lbp_features(img)
            combined_features = np.hstack((hog_features, lbp_features))
            X.append(combined_features)
            y.append(1)

    for img_name in os.listdir(no_mask_path):
        img = cv2.imread(os.path.join(no_mask_path, img_name))
        if img is not None:
            hog_features = extract_hog_features(img)
            lbp_features = extract_lbp_features(img)
            combined_features = np.hstack((hog_features, lbp_features))
            X.append(combined_features)
            y.append(0)

    return np.array(X), np.array(y)

# Preprocessing the dataset
X, y = load_dataset()
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# SVM Classifier
svm_model = SVC(kernel='rbf', C=10, gamma='scale')
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)

# Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=5, class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

# Neural Network
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

input_dim = X_train.shape[1]
model = SimpleNN(input_dim)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the neural network with learning rate scheduling
num_epochs = 100
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    scheduler.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate the neural network
model.eval()
with torch.no_grad():
    nn_preds = model(X_test_tensor)
    nn_preds = (nn_preds >= 0.5).float().numpy().flatten()

# Results
print("\nSVM Classifier Report:")
print(classification_report(y_test, svm_preds))
print("SVM Accuracy:", accuracy_score(y_test, svm_preds))

print("\nRandom Forest Classifier Report:")
print(classification_report(y_test, rf_preds))
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))

print("\nNeural Network Classifier Report:")
print(classification_report(y_test, nn_preds))
print("Neural Network Accuracy:", accuracy_score(y_test, nn_preds))

Epoch [10/100], Loss: 0.2709
Epoch [20/100], Loss: 0.1131
Epoch [30/100], Loss: 0.0557
Epoch [40/100], Loss: 0.0226
Epoch [50/100], Loss: 0.0111
Epoch [60/100], Loss: 0.0069
Epoch [70/100], Loss: 0.0041
Epoch [80/100], Loss: 0.0036
Epoch [90/100], Loss: 0.0026
Epoch [100/100], Loss: 0.0030

SVM Classifier Report:
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       386
           1       0.93      0.95      0.94       433

    accuracy                           0.94       819
   macro avg       0.94      0.94      0.94       819
weighted avg       0.94      0.94      0.94       819

SVM Accuracy: 0.938949938949939

Random Forest Classifier Report:
              precision    recall  f1-score   support

           0       0.96      0.85      0.90       386
           1       0.88      0.97      0.92       433

    accuracy                           0.91       819
   macro avg       0.92      0.91      0.91       819
weighted avg       0